In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import joblib
import logging

# ------------------------------------------------------------------------
# LOGGING & CONFIGURATION
# ------------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.StreamHandler()]
)
LOGGER = logging.getLogger(__name__)

CONFIG = {
    "csv_path": "/kaggle/input/tem-dataset/city_base_Tem.csv",  # <-- Update to your CSV
    "output_dir": "forecasts",
    "model_dir": "models",
    "initial": "365 days",
    "period": "180 days",
    "horizon": "90 days",
    "max_lag": 3,
    "prophet_param_grid": {
        "changepoint_prior_scale": [0.05, 0.1, 0.2],
        "seasonality_mode": ["additive", "multiplicative"]
    },
    "rf_param_dist": {
        "n_estimators": [50, 100, 200],
        "max_depth": [3, 5, 7, 10, None],
        "min_samples_split": [2, 5, 10],
        "max_features": ["sqrt", "log2", None]
    },
    "n_iter_rf_random_search": 10,
    "cv_splits": 3,
    "random_state": 42,
    "target_year": 2026,
    # We will forecast each of these three columns in a loop
    "temp_columns": [
        "temperature_2m_mean (°C)",
        "temperature_2m_max (°C)",
        "temperature_2m_min (°C)"
    ]
}

os.makedirs(CONFIG["model_dir"], exist_ok=True)
os.makedirs(CONFIG["output_dir"], exist_ok=True)

# ------------------------------------------------------------------------
# DATA LOADING FUNCTION
# ------------------------------------------------------------------------
def load_data_multi_city(csv_path: str) -> pd.DataFrame:
    """
    Loads CSV with columns:
      city_name, date, temperature_2m_mean (°C), temperature_2m_max (°C), temperature_2m_min (°C)
    Renames 'date' -> 'ds' so Prophet can recognize it, but does NOT pick any single column as 'y' yet.
    We'll do that when we loop over each target column.
    """
    LOGGER.info(f"Loading data from {csv_path} ...")
    if not os.path.isfile(csv_path):
        raise FileNotFoundError(f"Could not find file: {csv_path}")

    df = pd.read_csv(csv_path)

    # Check required columns
    required_cols = [
        'city_name', 
        'date', 
        'temperature_2m_mean (°C)', 
        'temperature_2m_max (°C)', 
        'temperature_2m_min (°C)'
    ]
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"CSV missing columns: {missing_cols}")

    # Convert date to datetime
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

    # Drop rows with invalid city_name or date
    null_mask = df['city_name'].isna() | df['date'].isna()
    if null_mask.any():
        LOGGER.warning(f"Dropping {null_mask.sum()} rows with NaNs in city_name or date.")
        df = df[~null_mask]

    # Rename 'date' -> 'ds' for Prophet convenience
    df.rename(columns={'date': 'ds'}, inplace=True)

    # Sort by ds ascending
    df.sort_values('ds', inplace=True)
    df.reset_index(drop=True, inplace=True)

    LOGGER.info(
        f"Data loaded. Shape: {df.shape}, Unique cities: {df['city_name'].nunique()}"
    )
    return df

# ------------------------------------------------------------------------
# MODELING & HELPER FUNCTIONS (Mostly same as before)
# ------------------------------------------------------------------------
def tune_prophet_hyperparams(df: pd.DataFrame, param_grid: dict, initial: str, period: str, horizon: str) -> Prophet:
    LOGGER.info("Tuning Prophet hyperparameters ...")
    best_model = None
    best_rmse = float('inf')
    
    for cps in param_grid["changepoint_prior_scale"]:
        for mode in param_grid["seasonality_mode"]:
            temp_model = Prophet(
                yearly_seasonality=True,
                weekly_seasonality=True,
                seasonality_mode=mode,
                changepoint_prior_scale=cps
            )
            temp_model.fit(df[['ds', 'y']])
            
            try:
                df_cv = cross_validation(temp_model, initial=initial, period=period, horizon=horizon)
                metrics = performance_metrics(df_cv)
                rmse = metrics['rmse'].mean()
            except Exception as e:
                LOGGER.warning(
                    f"[Prophet CV] Failed for CPS={cps}, Mode={mode}, error={e}. "
                    "Assigning large RMSE=9999999 to this combination."
                )
                rmse = 9999999
            
            LOGGER.info(f"Hyperparams => CPS={cps}, Mode={mode}, Mean RMSE={rmse:.3f}")
            if rmse < best_rmse:
                best_rmse = rmse
                best_model = temp_model
    
    if best_model is None:
        LOGGER.warning("All cross validation attempts failed; using default Prophet model.")
        best_model = Prophet().fit(df[['ds','y']])
    
    LOGGER.info(f"Best Prophet hyperparams => RMSE={best_rmse:.3f}")
    return best_model

def prophet_cross_val_evaluation(model, initial, period, horizon):
    LOGGER.info("Running Prophet cross-validation ...")
    try:
        df_cv = cross_validation(model, initial=initial, period=period, horizon=horizon)
        df_metrics = performance_metrics(df_cv)
        LOGGER.info(f"Prophet CV metrics:\n{df_metrics.head()}")
        return df_cv, df_metrics
    except Exception as e:
        LOGGER.warning(f"Prophet cross-validation failed: {e}")
        return pd.DataFrame(), pd.DataFrame()

def build_residual_dataset(df: pd.DataFrame, prophet_model: Prophet) -> pd.DataFrame:
    LOGGER.info("Creating in-sample forecast to compute residuals ...")
    # Predict only the historical range
    in_sample = prophet_model.make_future_dataframe(periods=0)
    forecast = prophet_model.predict(in_sample)
    df_merged = pd.merge(df, forecast[['ds','yhat']], on='ds', how='left')
    df_merged['residual'] = df_merged['y'] - df_merged['yhat']
    return df_merged

def create_lag_features(df_res: pd.DataFrame, target_col='residual', max_lag=3) -> pd.DataFrame:
    LOGGER.info("Creating lag features for the residual ...")
    df_lag = df_res.copy()
    df_lag['day_of_year'] = df_lag['ds'].dt.dayofyear
    df_lag['day_of_week'] = df_lag['ds'].dt.dayofweek
    for lag in range(1, max_lag+1):
        df_lag[f'{target_col}_lag_{lag}'] = df_lag[target_col].shift(lag)
    df_lag.dropna(inplace=True)
    return df_lag

def evaluate_model_predictions(y_true, y_pred, model_name="Model"):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    if np.all(y_true != 0):
        mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    else:
        mape = np.nan
    r2 = r2_score(y_true, y_pred)
    LOGGER.info(
        f"{model_name} => MAE={mae:.3f}, RMSE={rmse:.3f}, MAPE={mape:.2f}%, R^2={r2:.3f}"
    )
    return {"mae": mae, "rmse": rmse, "mape": mape, "r2": r2}

def tune_residual_model(df_supervised: pd.DataFrame, param_dist: dict, n_iter: int,
                        cv_splits: int, random_state: int, target_col='residual'):
    LOGGER.info("Training RandomForest on residual with RandomizedSearchCV ...")
    feature_cols = [c for c in df_supervised.columns if 'lag_' in c or 'day_of_' in c]
    X = df_supervised[feature_cols]
    y = df_supervised[target_col]
    
    tscv = TimeSeriesSplit(n_splits=cv_splits)
    rf = RandomForestRegressor(random_state=random_state)
    
    randomized_search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_dist,
        n_iter=n_iter,
        cv=tscv,
        scoring='neg_mean_squared_error',
        random_state=random_state,
        n_jobs=-1
    )
    randomized_search.fit(X, y)
    
    best_rf_model = randomized_search.best_estimator_
    best_params = randomized_search.best_params_
    best_score = -randomized_search.best_score_
    LOGGER.info(f"Best RF Params => {best_params}, MSE={best_score:.2f}")
    
    y_pred = best_rf_model.predict(X)
    _ = evaluate_model_predictions(y, y_pred, model_name="RandomForest (Residual)")
    
    return best_rf_model, feature_cols

def iterative_residual_prediction(model, df_history, future_dates, feature_cols, max_lag=3) -> pd.DataFrame:
    df_temp = df_history.copy()
    predictions = []
    for dt in future_dates:
        last_row = df_temp.iloc[-1].copy()
        # Shift residual forward
        for lag in range(1, max_lag+1):
            if lag == 1:
                val = last_row['residual']
            else:
                val = last_row[f'residual_lag_{lag-1}']
            last_row[f'residual_lag_{lag}'] = val
        
        last_row['ds'] = dt
        last_row['day_of_year'] = dt.day_of_year
        last_row['day_of_week'] = dt.day_of_week
        
        X_new = pd.DataFrame([last_row[feature_cols]], columns=feature_cols)
        resid_pred = model.predict(X_new)[0]
        
        new_row = last_row.copy()
        new_row['ds'] = dt
        new_row['residual'] = resid_pred
        predictions.append([dt, resid_pred])
        
        df_temp = pd.concat([df_temp, pd.DataFrame([new_row])], ignore_index=True)
    
    df_pred = pd.DataFrame(predictions, columns=['ds','residual_pred'])
    return df_pred

def create_hybrid_forecast(prophet_model, rf_model, df_supervised, feature_cols, target_year, max_lag):
    LOGGER.info(f"Creating hybrid forecast for {target_year} ...")
    start_date = f"{target_year}-01-01"
    end_date = f"{target_year}-12-31"
    last_ds = df_supervised['ds'].max()
    days_needed = (pd.to_datetime(end_date) - last_ds).days
    if days_needed < 1:
        raise ValueError(f"Requested year {target_year} <= last training date {last_ds}.")

    # Prophet forecast
    future_df = prophet_model.make_future_dataframe(periods=days_needed)
    prophet_forecast = prophet_model.predict(future_df)
    prophet_future = prophet_forecast[
        (prophet_forecast['ds'] >= start_date) & (prophet_forecast['ds'] <= end_date)
    ].copy()

    # Residual forecast
    df_history_for_res = df_supervised.iloc[[-1]].copy()
    future_dates = pd.to_datetime(prophet_future['ds'].unique())
    df_future_resid = iterative_residual_prediction(
        rf_model, df_history_for_res, future_dates, feature_cols, max_lag
    )

    # Merge
    hybrid_df = prophet_future.merge(df_future_resid, on='ds', how='left')
    hybrid_df['residual_pred'].fillna(0, inplace=True)
    hybrid_df['yhat_hybrid'] = hybrid_df['yhat'] + hybrid_df['residual_pred']
    return hybrid_df

# ------------------------------------------------------------------------
# PLOTTING FUNCTION (Slightly generalized)
# ------------------------------------------------------------------------
def plot_full_forecast_and_save(city: str, forecast_df: pd.DataFrame, target_year: int, var_name: str):
    """
    Plots the forecast for the given 'var_name' (e.g., 'mean', 'max', or 'min').
    We show:
      - Observed history (black)
      - Prophet baseline (blue dashed)
      - Hybrid forecast (red)
      - Monthly average bar of the hybrid forecast
    """
    LOGGER.info(f"Plotting {var_name} forecast for city={city}, year={target_year}")
    # Load full data to get historical "y"
    df_all = load_data_multi_city(CONFIG["csv_path"])
    df_city = df_all[df_all["city_name"] == city].copy()
    if df_city.empty:
        LOGGER.warning(f"No historical data found for city: {city}")
        return

    # We assume the user replaced the actual data with 'y' for training, 
    # so let's do the same here for consistency in plotting:
    # We'll do that if you stored 'y' in the same column. Otherwise, 
    # you'd store the actual col differently. For minimal changes, let's do a fallback:
    # For "mean" -> use "temperature_2m_mean (°C)"
    # For "max"  -> use "temperature_2m_max (°C)"
    # For "min"  -> use "temperature_2m_min (°C)"
    # You can adapt as needed if your actual data differs.

    # Convert ds to datetime
    if not pd.api.types.is_datetime64_any_dtype(forecast_df['ds']):
        forecast_df['ds'] = pd.to_datetime(forecast_df['ds'])
    
    # Filter for the target year
    forecast_year = forecast_df[forecast_df['ds'].dt.year == target_year].copy()
    if forecast_year.empty:
        LOGGER.warning(f"No forecast data found for {var_name} in {target_year}.")
        return

    forecast_start = forecast_year['ds'].min()
    observed = df_city[df_city['ds'] < forecast_start].copy()

    # If you want to plot the actual historical col, do something like:
    if var_name == "mean":
        if "temperature_2m_mean (°C)" in observed.columns:
            observed.rename(columns={"temperature_2m_mean (°C)": "y"}, inplace=True)
        else:
            observed["y"] = np.nan
    elif var_name == "max":
        if "temperature_2m_max (°C)" in observed.columns:
            observed.rename(columns={"temperature_2m_max (°C)": "y"}, inplace=True)
        else:
            observed["y"] = np.nan
    elif var_name == "min":
        if "temperature_2m_min (°C)" in observed.columns:
            observed.rename(columns={"temperature_2m_min (°C)": "y"}, inplace=True)
        else:
            observed["y"] = np.nan

    fig, axes = plt.subplots(2, 1, figsize=(12, 8))

    # --- Top subplot ---
    ax1 = axes[0]
    ax1.set_title(f"Hybrid {var_name.title()} Temperature Forecast for {city} ({target_year})")

    # Observed (black)
    ax1.plot(observed['ds'], observed['y'], color='black', label='Observed (History)')

    # Prophet baseline (blue dashed)
    if 'yhat_lower' in forecast_year.columns and 'yhat_upper' in forecast_year.columns:
        ax1.fill_between(
            forecast_year['ds'],
            forecast_year['yhat_lower'],
            forecast_year['yhat_upper'],
            color='gray', alpha=0.2,
            label='Prophet Uncertainty'
        )
    ax1.plot(
        forecast_year['ds'],
        forecast_year['yhat'],
        linestyle='--',
        color='blue',
        label='Prophet Baseline'
    )

    # Hybrid (red)
    ax1.plot(
        forecast_year['ds'],
        forecast_year['yhat_hybrid'],
        color='red',
        label='Hybrid Forecast'
    )

    ax1.set_ylabel(f"{var_name.title()} Temp (°C)")
    ax1.legend(loc='best')

    # --- Bottom subplot: monthly average of the hybrid forecast
    ax2 = axes[1]
    ax2.set_title(f"Monthly Average {var_name.title()} Temperature ({target_year})")
    forecast_year['month'] = forecast_year['ds'].dt.month
    monthly_avg = (
        forecast_year.groupby('month')['yhat_hybrid']
        .mean()
        .reset_index(name='avg_temp')
    )
    ax2.bar(monthly_avg['month'], monthly_avg['avg_temp'])
    ax2.set_xlabel("Month")
    ax2.set_ylabel(f"Avg {var_name.title()} Temp (°C)")
    ax2.set_xticks(range(1,13))
    ax2.set_xticklabels([f"{m:02d}" for m in range(1,13)])
    ax2.set_xlim([0.5,12.5])

    plt.tight_layout()
    plot_filename = os.path.join(
        CONFIG["output_dir"],
        f"{city}_{var_name}_forecast_{target_year}_hybrid.png"
    )
    plt.savefig(plot_filename, dpi=150)
    LOGGER.info(f"Chart saved to {plot_filename}")
    plt.show()

# ------------------------------------------------------------------------
# TRAINING LOOP FOR ALL CITIES & 3 COLUMNS
# ------------------------------------------------------------------------
df_all = load_data_multi_city(CONFIG["csv_path"])
unique_cities = df_all['city_name'].unique()
LOGGER.info(f"Found {len(unique_cities)} cities: {unique_cities}")

for city in unique_cities:
    df_city = df_all[df_all['city_name'] == city].copy()
    LOGGER.info(f"\n===== Processing city: {city} | Data shape: {df_city.shape} =====")
    
    # If the city has fewer than 2 rows, skip
    if df_city.shape[0] < 2:
        LOGGER.warning(f"Not enough data for city {city}. Skipping.")
        continue

    # For each of the three columns (mean, max, min), do the same pipeline
    for temp_col in CONFIG["temp_columns"]:
        LOGGER.info(f"\n--- Forecasting column '{temp_col}' for city={city} ---")

        # 1) Rename that column to 'y'
        if temp_col not in df_city.columns:
            LOGGER.warning(f"Column '{temp_col}' not found for city={city}. Skipping.")
            continue

        df_for_prophet = df_city[['ds', temp_col]].copy()
        df_for_prophet.rename(columns={temp_col: 'y'}, inplace=True)

        # Drop rows with missing y
        df_for_prophet.dropna(subset=['y'], inplace=True)
        if df_for_prophet.shape[0] < 2:
            LOGGER.warning(f"Not enough valid rows in '{temp_col}' for city={city}. Skipping.")
            continue

        # 2) Train Prophet
        prophet_model = tune_prophet_hyperparams(
            df_for_prophet,
            CONFIG["prophet_param_grid"],
            CONFIG["initial"],
            CONFIG["period"],
            CONFIG["horizon"]
        )

        # Optional cross validation
        _, _ = prophet_cross_val_evaluation(
            prophet_model,
            CONFIG["initial"],
            CONFIG["period"],
            CONFIG["horizon"]
        )

        # 3) Build residual dataset
        df_residual = build_residual_dataset(df_for_prophet, prophet_model)
        df_supervised = create_lag_features(df_residual, target_col='residual', max_lag=CONFIG["max_lag"])
        if df_supervised.empty:
            LOGGER.warning(f"Residual dataset empty after lagging for {temp_col} in city={city}.")
            continue

        # 4) RandomForest on residual
        rf_model, feature_cols = tune_residual_model(
            df_supervised,
            CONFIG["rf_param_dist"],
            CONFIG["n_iter_rf_random_search"],
            CONFIG["cv_splits"],
            CONFIG["random_state"]
        )

        # 5) Hybrid forecast
        target_year = CONFIG["target_year"]
        try:
            hybrid_forecast = create_hybrid_forecast(
                prophet_model,
                rf_model,
                df_supervised,
                feature_cols,
                target_year,
                CONFIG["max_lag"]
            )
        except ValueError as e:
            LOGGER.warning(f"Could not create forecast for city={city}, col={temp_col}: {e}")
            continue

        # 6) Plot & save
        # We pick a label for the variable (like 'mean','max','min')
        if "mean" in temp_col.lower():
            var_name = "mean"
        elif "max" in temp_col.lower():
            var_name = "max"
        elif "min" in temp_col.lower():
            var_name = "min"
        else:
            var_name = "temp"

        plot_full_forecast_and_save(city, hybrid_forecast, target_year, var_name)

        # Save the forecast CSV
        safe_col_name = temp_col.replace(" ","_").replace("(","").replace(")","")
        forecast_filename = os.path.join(
            CONFIG["output_dir"],
            f"{city}_{safe_col_name}_{target_year}_hybrid.csv"
        )
        hybrid_forecast.to_csv(forecast_filename, index=False)
        LOGGER.info(f"Forecast for {temp_col} saved to {forecast_filename}")

        # 7) Save the models
        model_prefix = safe_col_name  # e.g. "temperature_2m_mean_°C"
        prophet_path = os.path.join(CONFIG["model_dir"], f"{city}_prophet_{model_prefix}.pkl")
        rf_path = os.path.join(CONFIG["model_dir"], f"{city}_rf_residual_{model_prefix}.pkl")

        joblib.dump(prophet_model, prophet_path)
        joblib.dump(rf_model, rf_path)
        LOGGER.info(f"Saved Prophet model to {prophet_path}")
        LOGGER.info(f"Saved RF model to {rf_path}")

# ------------------------------------------------------------------------
# PREDICTION FUNCTION (Slightly adapted for multiple columns)
# ------------------------------------------------------------------------
def predict_forecast(city: str, target_year: int, temp_col: str) -> pd.DataFrame:
    """
    Loads saved Prophet + RF models (specific to 'temp_col') for the city,
    rebuilds the residual dataset, and produces the hybrid forecast for target_year.
    """
    df_all = load_data_multi_city(CONFIG["csv_path"])
    df_city = df_all[df_all["city_name"] == city].copy()
    if df_city.shape[0] < 2:
        raise ValueError(f"Not enough data for city='{city}'")

    # Convert the chosen column -> 'y'
    if temp_col not in df_city.columns:
        raise ValueError(f"Column '{temp_col}' not found for city='{city}' in data.")

    df_for_prophet = df_city[['ds', temp_col]].copy()
    df_for_prophet.rename(columns={temp_col: 'y'}, inplace=True)
    df_for_prophet.dropna(subset=['y'], inplace=True)
    if df_for_prophet.shape[0] < 2:
        raise ValueError(f"No valid data in '{temp_col}' for city='{city}'.")

    # Load the correct models
    safe_col_name = temp_col.replace(" ","_").replace("(","").replace(")","")
    prophet_path = os.path.join(CONFIG["model_dir"], f"{city}_prophet_{safe_col_name}.pkl")
    rf_path = os.path.join(CONFIG["model_dir"], f"{city}_rf_residual_{safe_col_name}.pkl")

    if not os.path.exists(prophet_path) or not os.path.exists(rf_path):
        raise FileNotFoundError(f"Model files for {temp_col} in city='{city}' not found.")

    prophet_model = joblib.load(prophet_path)
    rf_model = joblib.load(rf_path)

    # Rebuild residual dataset
    df_residual = build_residual_dataset(df_for_prophet, prophet_model)
    df_supervised = create_lag_features(df_residual, target_col='residual', max_lag=CONFIG["max_lag"])
    if df_supervised.empty:
        raise ValueError(f"No valid residual data after lagging for city='{city}' & col='{temp_col}'.")

    feature_cols = [c for c in df_supervised.columns if 'lag_' in c or 'day_of_' in c]

    # Hybrid forecast
    forecast = create_hybrid_forecast(
        prophet_model,
        rf_model,
        df_supervised,
        feature_cols,
        target_year,
        CONFIG["max_lag"]
    )
    return forecast
